loader = PyPDFLoader("C:/Users/matan.s/Desktop/python/AI/GPT4Aall/2022_Annual_Report.pdf")

In [1]:
# !pip install -Uqqq pip --progress-bar off
# !pip install -qqq langchain==0.0.173 --progress-bar off
# !pip install -qqq chromadb==0.3.23 --progress-bar off
# !pip install -qqq pypdf==3.8.1 --progress-bar off
# !pip install -qqq pygpt4all==1.1.0 --progress-bar off
# !pip install -qqq pdf2image==1.16.3 --progress-bar off

In [2]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path

## Load Data

In [3]:
loader = PyPDFLoader("C:/Users/matan.s/Desktop/python/AI/GPT4Aall/2022_Annual_Report.pdf")

In [4]:
documents = loader.load_and_split()

In [5]:
len(documents)

115

In [6]:
print(documents[0].page_content)

1 Dear shareholders, colleagues, customers, and partners:  
We are living through a period of historic economic, societal, and geopolitical change. The world in 2022 looks nothing like 
the world in 2019. As I write this, inflation is at a 40 -year high, supply chains are stretched, and the war in Ukraine is 
ongoing.  At the same time, we are entering a technological era with the potential to power awesome advancements 
across every sector of our economy and society. As the world’s largest software company, this places us at a historic 
intersection of opportunity and re sponsibility to the world around us.  
Our mission to empower every person and every organization on the planet to achieve more has never been more 
urgent or more necessary. For all the uncertainty in the world, one thing is clear: People and organizations in every 
industry are increasingly looking to digital technology to overcome today’s challenges and emerge stronger. And no 
company is better positioned to help 

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(documents)

In [8]:
len(texts)

359

In [9]:
print(texts[0].page_content)

1 Dear shareholders, colleagues, customers, and partners:  
We are living through a period of historic economic, societal, and geopolitical change. The world in 2022 looks nothing like 
the world in 2019. As I write this, inflation is at a 40 -year high, supply chains are stretched, and the war in Ukraine is 
ongoing.  At the same time, we are entering a technological era with the potential to power awesome advancements 
across every sector of our economy and society. As the world’s largest software company, this places us at a historic 
intersection of opportunity and re sponsibility to the world around us.  
Our mission to empower every person and every organization on the planet to achieve more has never been more 
urgent or more necessary. For all the uncertainty in the world, one thing is clear: People and organizations in every 
industry are increasingly looking to digital technology to overcome today’s challenges and emerge stronger. And no 
company is better positioned to help 

## Create Embeddings

In [10]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [11]:
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

Using embedded DuckDB with persistence: data will be stored in: db


## Create Chain

In [12]:
model_n_ctx = 1000
model_path = "./ggml-gpt4all-j-v1.3-groovy.bin"
llm = GPT4All(model=model_path, n_ctx=1000, backend="gptj", verbose=False)

In [13]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    verbose=False,
)

## Ask Questions

In [14]:
%%time
res = qa(
    "How much is the dividend per share during during 2022? Extract it from the text."
)

[2023-06-22 08:44:11,744] {_common.py:120} ERROR - Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='app.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1007)'))))
CPU times: total: 14min 28s
Wall time: 3min 38s


In [20]:
res

{'query': 'How much is the dividend per share during during 2022? Extract it from the text.',
 'result': '\n\nI cannot answer this question without knowing the year or period for which the dividend is being referred to.',
 'source_documents': [Document(page_content='Year ended June  30, 2022  2021  2020  \n        \nDividends per share (quarterly amounts)  $   0.56 –  0.62  $   0.51 – 0.56  $ 0.46 – 0.51  \nInterest rates   0.03%  – 3.6%    0.01%  – 1.5%    0.1%  – 2.2%', metadata={'source': 'C:/Users/matan.s/Desktop/python/AI/GPT4Aall/2022_Annual_Report.pdf', 'page': 80}),
  Document(page_content='Total      $ 2.24  $ 16,871             \nThe dividend declared on June  14, 2022 was included in other current liabilities as of June  30, 2022.', metadata={'source': 'C:/Users/matan.s/Desktop/python/AI/GPT4Aall/2022_Annual_Report.pdf', 'page': 10}),
  Document(page_content='Balance, beginning of period   1,822    3,186    (340) \nOther comprehensive income (loss)   (6,500 )  (1,374 )  3,52

In [16]:
# print(res["result"])

In [17]:
# %%time
# prompt = f"""How much is the investment amount in Microsoft on 6/22? Extract the answer from the text."""
# res = qa(prompt.strip())

In [18]:
# #this cell is vanilen run
# %%time
# prompt = f"""How much is the investment amount in Microsoft on 6/22? Extract the answer from the text."""
# res = qa(prompt.strip())
# CPU times: user 12min 41s, sys: 9.47 s, total: 12min 51s
# Wall time: 3min 29s

In [19]:
# print(res["result"])

[2023-06-22 08:44:20,380] {_common.py:105} INFO - Backing off send_request(...) for 0.0s (requests.exceptions.SSLError: HTTPSConnectionPool(host='app.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1007)'))))
[2023-06-22 08:44:20,559] {_common.py:105} INFO - Backing off send_request(...) for 1.8s (requests.exceptions.SSLError: HTTPSConnectionPool(host='app.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1007)'))))
[2023-06-22 08:44:22,556] {_common.py:105} INFO - Backing off send_request(...) for 3.1s (requests.exceptions.SSLError: HTTPSConnectionPool(host='app.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by S

## References

- [GPT4All](https://gpt4all.io/)
- [MICROSOFT 2022 ANNUAL REPORT](https://www.microsoft.com/investor/reports/ar22/download-center/)